In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

# 크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager

# 크롬 옵션 설정
chrome_options = Options()
# 사용자 에이전트 정보 설정
user_agent = f"{'user_agnet'}"
chrome_options.add_argument('user-agent=' + user_agent)
# 메모리가 부족해서 에러 발생 막기
chrome_options.add_argument("--disable-dev-shm-usage")
# 이미지 로딩 하지 않기
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
# 브라우저가 꺼지지 않도록 설정
chrome_options.add_experimental_option("detach", True)

# 웹드라이버 연결 함수
def connect_url(url, head=None):
    # 크롬드라이버 다운로드 및 자동 업데이트
    service = Service(executable_path=ChromeDriverManager().install())
    # headless 모드 사용여부에 따라 설정 변경
    if head == "less":
        chrome_options.add_argument('headless')
    else:
        pass
    try:
        # 웹드라이버 객체 생성 후 url 접속
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.get(url)
        return driver
    except Exception as e:
        # 에러 발생 시 예외 처리
        print(f"접속 실패: {e}")
        return


In [26]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# FomcDate 클래스
class FomcDate:

    def __init__(self, url:str):
        # 생성자에서 url을 인자로 받아 객체 변수에 저장
        self.url = url

    # FOMC 날짜 데이터를 크롤링하여 리스트로 반환하는 메서드
    def get_fomc_data(self):
        # 웹드라이버 연결
        driver = connect_url(self.url)
        try:
            # 팝업창 닫기 버튼 클릭
            driver.find_element(By.CSS_SELECTOR, "#PromoteSignUpPopUp > div.right > i").click()
        except:
            pass
        # 페이지에 더 보기 버튼이 나타날 때까지 무한 반복
        while True:
            try:
                # 더 보기 버튼 클릭
                add_button = driver.find_element(By.XPATH, '//*[@id="showMoreHistory1738"]/a')
                add_button.click()
                try:
                    # 팝업창 닫기 버튼 클릭
                    x_button = driver.find_element(By.CSS_SELECTOR, "#PromoteSignUpPopUp > div.right > i")
                    x_button.click()
                except:
                    pass
            except:
                break
        # 크롤링한 페이지의 소스코드를 파싱하여 BeautifulSoup 객체 생성
        soup = bs(driver.page_source, 'html.parser')
        # release_date 리스트에 있는 날짜 데이터를 추출하여 fomc_date_list에 저장
        fomc_date_list = []
        release_date = soup.select("div.historyTab > table > tbody > tr > td.left")
        for fomc_date in release_date:
            data = {"date":fomc_date.string.replace("년 ","-").replace("월 ","-").replace("일","")}
            fomc_date_list.append(data)
        # 웹드라이버 종료
        driver.close()
        # fomc_date_list 반환
        return fomc_date_list

    # FOMC 날짜 데이터를 전처리하고 csv 파일로 저장한 뒤, DataFrame을 반환하는 메서드
    def fomc_preprocessing(self):
        # get_fomc_data() 메서드를 호출하여 FOMC 날짜 데이터를 리스트로 얻는다.
        df = pd.DataFrame(self.get_fomc_data())
        # 'symbol' column을 추가하여 'FOMC' 값을 할당
        df.loc[:,"symbol"] = "FOMC"
        # 날짜순으로 정렬
        df.sort_values('date', inplace=True)
        # 'date' column의 불필요한 공백을 제거
        df['date'] = df['date'].str.strip()
        # 'date' column을 인덱스로 설정
        df.set_index("date", inplace=True)
        # 전처리된 DataFrame을 csv 파일로 저장
        df.to_csv('./data/fomc_date.csv')
        # 전처리된 DataFrame을 반환
        return df

if __name__ == "__main__":
    # FOMC 날짜 데이터 크롤링을 위한 url
    url = "https://kr.investing.com/economic-calendar/fed-chair-powell-speaks-1738"
    # FomcDate 클래스 객체 생성
    fomc = FomcDate(url)
    # FOMC 날짜 데이터를 전처리하여 DataFrame으로 반환
    fomc_date = fomc.fomc_preprocessing()
    # 전처리된 DataFrame 출력
    print(fomc_date)


           symbol
fomc_date        
2018-04-07   FOMC
2018-05-08   FOMC
2018-05-25   FOMC
2018-06-20   FOMC
2018-08-24   FOMC
...           ...
2022-08-26   FOMC
2022-09-08   FOMC
2022-09-24   FOMC
2022-09-28   FOMC
2022-12-01   FOMC

[72 rows x 1 columns]


In [4]:
import json
import pandas as pd
from bs4 import BeautifulSoup as bs
from tqdm import tqdm

# NasdaqPrice 클래스
class NasdaqPrice:

    # 생성자에서 크롤링할 페이지 수와 브라우저 실행 여부를 인자로 받아 객체 변수에 저장
    def __init__(self, pages:int, head:str):
        self.pages = pages
        self.head = head

    # 네이버 금융에서 NASDAQ 종가 데이터를 크롤링하여 리스트로 반환하는 메서드
    def get_nasdaq_data(self):
        result = []
        # 페이지 수만큼 반복
        for page in tqdm(range(1, self.pages)):
            # NASDAQ 종가 데이터를 크롤링할 url을 생성
            url = f"https://finance.naver.com/world/worldDayListJson.naver?symbol=NAS@IXIC&fdtc=0&page={page}"
            # 생성한 url을 사용하여 웹드라이버를 연결
            driver = connect_url(url, self.head)
            # 페이지의 소스코드를 파싱하여 BeautifulSoup 객체를 생성
            soup = bs(driver.page_source, 'html.parser')
            # pre 태그에서 텍스트 데이터를 추출
            nasdaq_text = soup.select("pre")[0].text
            # 추출한 텍스트 데이터를 json.loads() 함수를 사용하여 리스트로 변환
            nasdaq_jsons = json.loads(nasdaq_text)
            # 변환한 리스트를 result 리스트에 추가
            result.extend(nasdaq_jsons)
        # 모든 페이지에서 크롤링한 데이터를 반환
        return result

    # NASDAQ 종가 데이터를 전처리하고 csv 파일로 저장한 뒤, DataFrame을 반환하는 메서드
    def nasdaq_preprocessing(self):
        # get_nasdaq_data() 메서드를 호출하여 네이버 금융에서 크롤링한 데이터를 리스트로 얻는다.
        result = self.get_nasdaq_data()
        # 리스트를 DataFrame으로 변환
        df = pd.DataFrame(result)
        # 'xymd', 'clos', 'symb' column만 선택
        df = df[['xymd','clos','symb']]
        # column 이름을 바꾼다.
        df.rename(columns={"xymd":"date", "clos":"nasdaq_price","symb":"symbol"}, inplace=True)
        # 'date' column을 datetime 형식으로 변환
        df['date'] = pd.to_datetime(df['date'])
        # 'date' column을 기준으로 오름차순으로 정렬
        df.sort_values('date', inplace=True)
        # 'symbol' column에 'NASDAQ' 값을 할당
        df.loc[:,'symbol'] = "NASDAQ"
        # 'date' column을 인덱스로 설정
        df.set_index("date", inplace=True)
        # 전처리된 DataFrame을 csv 파일로 저장
        df.to_csv("./data/nasdaq_price.csv")
        # 전처리된 DataFrame을 반환
        return df


if __name__ == "__main__":
    # 크롤링할 페이지 수와 브라우저 실행 여부를 설정
    pages = 5
    head = "less"
    # NasdaqPrice 클래스 객체를 생성
    nasdaq = NasdaqPrice(pages, head)
    # get_nasdaq_data() 메서드를 호출하여 네이버 금융에서 NASDAQ 종가 데이터를 크롤링한 뒤, 리스트로 반환
    nasdaq_price = nasdaq.get_nasdaq_data()
    # 반환된 리스트를 DataFrame으로 변환
    df = pd.DataFrame(nasdaq_price)
    # 전처리된 DataFrame을 출력
    print(df)



100%|██████████| 4/4 [00:12<00:00,  3.08s/it]


        symb      xymd      open      high       low      clos    diff  rate  \
0   NAS@IXIC  20230113  10906.37  11084.01  10900.16  11079.16   78.05  0.71   
1   NAS@IXIC  20230112  10969.26  11027.75  10797.12  11001.11   69.44  0.64   
2   NAS@IXIC  20230111  10794.99  10932.44  10762.73  10931.67  189.04  1.76   
3   NAS@IXIC  20230110  10607.72  10743.67  10589.60  10742.63  106.98  1.01   
4   NAS@IXIC  20230109  10662.11  10807.26  10619.12  10635.65   66.36  0.63   
5   NAS@IXIC  20230106  10363.96  10604.14  10265.04  10569.29  264.05  2.56   
6   NAS@IXIC  20230105  10390.31  10393.22  10295.25  10305.24 -153.52 -1.47   
7   NAS@IXIC  20230104  10467.82  10515.22  10337.64  10458.76   71.77  0.69   
8   NAS@IXIC  20230103  10562.06  10613.06  10309.16  10386.99  -79.49 -0.76   
9   NAS@IXIC  20221230  10368.37  10468.31  10324.70  10466.48  -11.61 -0.11   
10  NAS@IXIC  20221229  10321.46  10502.08  10301.06  10478.09  264.80  2.59   
11  NAS@IXIC  20221228  10339.20  10414.

In [7]:
import requests as req
# xml형식의 데이터를 dict형식으로 파싱함
import xmltodict
import json
import pandas as pd

class KospiPrice:
    
    
    def __init__(self, symbol:str, count:int):
        # KospiPrice 클래스를 초기화
        # symbol은 주식 종목 코드를, count는 조회할 데이터 개수를 입력
        self.symbol = symbol
        self.count = count
    
    # 네이버 금융에서 일별 주가 데이터를 조회하는 함수
    def get_kospi_data(self):
        # API 호출 URL을 생성합니다.
        url = f"https://fchart.stock.naver.com/sise.nhn?symbol={self.symbol}&timeframe=day&count={self.count}&requestType=0"
        # API 호출을 수행
        kospi_get = req.get(url)
        # 조회한 데이터를 xml 형식에서 dict 형식으로 변환
        kospi_data = xmltodict.parse(kospi_get.text)
        # 변환된 dict 형식의 데이터를 json 형식으로 변환
        kospi_dump = json.dumps(kospi_data, ensure_ascii=False)
        kospi_json = json.loads(kospi_dump)
        # 필요한 데이터만 추출
        kospi_data = kospi_json['protocol']['chartdata']['item']
        # 추출한 데이터를 반환
        return kospi_data

    def kospi_preprocessing(self):
        # 코스피 데이터 가져오기
        kospi_data = self.get_kospi_data()
        # 판다스 데이터프레임 생성
        df = pd.DataFrame(kospi_data)
        # '@data' 열을 분리하여 여러 개의 열로 만듦
        df = df['@data'].str.split("|", expand=True)
        # 열 이름 지정
        df.columns=['날짜','시가','고가','저가','종가','거래량']
        # '날짜' 열을 datetime 형식으로 변환
        df['날짜'] = pd.to_datetime(df['날짜'])
        # '날짜'와 '종가' 열만 있는 새로운 데이터프레임 생성
        df_close_price = df[['날짜','종가']]
        # 열 이름 변경
        df_close_price.rename(columns={"날짜":"date","종가":"kospi_price"}, inplace=True)
        # 'symbol' 열 추가
        df_close_price.loc[:,'symbol'] = 'KOSPI'
        # 'date' 열을 인덱스로 설정
        df_close_price.set_index("date", inplace=True)
        # 데이터프레임을 CSV 파일로 저장
        df_close_price.to_csv('./data/kospi_price.csv')
        # 데이터프레임 반환
        return df_close_price

if __name__ == "__main__":
    # 심볼과 카운트 변수 설정
    symbol = "KOSPI"
    count = 5
    # KospiPrice 클래스의 인스턴스 생성
    kospi = KospiPrice(symbol, count)
    # 코스피 데이터 가져오기
    kospi_price = kospi.get_kospi_data()
    # 판다스 데이터프레임 생성
    df = pd.DataFrame(kospi_price)
    # 데이터프레임 출력
    print(df)

                                             @data
0  20230111|2364.05|2369.66|2350.36|2359.53|369010
1    20230112|2376.72|2377.8|2358.33|2365.1|580433
2  20230113|2377.34|2397.01|2375.78|2386.09|862230
3  20230116|2390.06|2410.22|2387.61|2399.86|517762
4   20230117|2400.92|2404.0|2374.65|2379.39|529415


In [5]:
# 필요한 라이브러리 불러오기
from datetime import datetime
import time
import pandas as pd
from tqdm import tqdm
import requests as req

class EthereumPrice:

    # 클래스를 시작일, 종료일 및 심볼과 함께 초기화
    def __init__(self, start_date, end_date, symbol):
        self.symbol = symbol
        self.start_date = start_date
        self.end_date = end_date

    # 주어진 심볼에 대한 코인 심볼 가져오는 함수
    def get_coin_symbol(self):
        # API 호출을 위한 URL 정의
        url = "https://api.binance.com/api/v3/ticker/price"
        # API 호출하고 응답 받기
        response= req.get(url)
        # 응답을 JSON 형식으로 변환
        response_js = response.json()
        # 응답에서 심볼 추출
        symbols = [x['symbol'] for x in response_js]
        # 주어진 심볼과 일치하는 심볼 찾기
        symbol = [x for x in symbols if self.symbol in x]
        # 심볼 반환
        return symbol


    def get_eth_data(self):
        # API 호출을 위한 URL 정의
        url = "https://api.binance.com/api/v3/klines"

        # 시작일과 종료일을 timestamp(ms) 형식으로 변환
        start = int(time.mktime(datetime.strptime(self.start_date + ' 00:00', '%Y-%m-%d %H:%M').timetuple())) * 1000
        end = int(time.mktime(datetime.strptime(self.end_date +' 23:59', '%Y-%m-%d %H:%M').timetuple())) * 1000

        # API 호출에 사용될 파라미터 정의
        params = {
            'symbol': self.get_coin_symbol()[0],  # get_coin_symbol 함수를 통해 해당 코인의 심볼을 가져옴
            'interval': '1d',
            'limit': 1000,
            'startTime': start,
            'endTime': end
        }

        # 데이터 저장할 리스트 초기화 및 tqdm 프로그래스바 초기화
        data = []
        time_bar = tqdm(total=100)

        # 시작일부터 종료일까지 1일 간격으로 데이터 가져오기
        while start < end:
            # 시작일 출력
            print(datetime.fromtimestamp(start // 1000))
            # API 호출에 사용될 파라미터 중 startTime 변경
            params['startTime'] = start
            # API 호출하고 응답 받기
            response = req.get(url, params=params)
            # 응답을 JSON 형식으로 변환
            response_jsons = response.json()
            # 응답이 없으면 루프 종료
            if not response_jsons:
                break
            # 데이터에 응답 결과 추가
            data.extend(response_jsons)
            # 다음 step으로 이동하기 위해 startTime 변경
            start = response_jsons[-1][0] + 60000
            # tqdm 프로그래스바 업데이트
            time_bar.update(1)
        time_bar.close()
        # 최종 데이터 반환
        return data
    
    # Ethereum 가격 데이터 가져오고 전처리하는 함수
    def eth_preprocessing(self):
        # Ethereum 가격 데이터 가져오기
        data = self.get_eth_data()
        # 가져온 데이터가 없으면 -1 반환
        if not data:
            print('해당 기간에 일치하는 데이터가 없습니다.')
            return -1
        # 데이터 프레임 생성
        columns = ['eth_date', 'Open_price', 'High_price', 'Low_price', 'eth_price', 'Volume', 'Close_time', 'Quote_asset_volume', 'Number_of_trades',
                    'Taker_buy_base_asset_volume', 'Taker_buy_quote_asset_volume', 'Unused_field']
        df = pd.DataFrame(data, columns=columns)
        new_df = df.copy()
        # 불필요한 열 제거
        new_df = df.drop(columns=['Open_price', 'High_price', 'Low_price', 'Volume',
                                    'Close_time','Quote_asset_volume','Number_of_trades',
                                    'Taker_buy_base_asset_volume','Taker_buy_quote_asset_volume','Unused_field'])
        # 날짜 형식 변환
        new_df['date'] = new_df.apply(lambda x:datetime.fromtimestamp(x['eth_date'] // 1000), axis=1).dt.strftime('%Y-%m-%d')
        # 코인 심볼 추가
        new_df['symbol'] = self.get_coin_symbol()[0]
        # 가격 열 데이터 타입 변환 및 소수점 이하 2자리까지 반올림
        new_df['eth_price'] = new_df['eth_price'].astype(float)
        new_df['eth_price'] = round(new_df['eth_price'], 2)
        # 날짜를 인덱스로 설정하고 CSV 파일로 저장
        new_df = new_df.set_index("date")
        new_df.to_csv("./data/eth_price.csv")
        # 전처리된 데이터프레임 반환
        return new_df


if __name__ == "__main__":
    # 현재 시간과 Ethereum 심볼, 시작일 및 종료일 지정
    now = datetime.now().strftime("%Y-%m-%d")
    symbol = 'ETHUSDT'
    start_date = '2012-04-06'
    end_date = now
    # EthereumPrice 클래스의 인스턴스 생성
    ethereum = EthereumPrice(start_date, end_date, symbol)
    # Ethereum 가격 데이터 전처리
    ethereum_price = ethereum.eth_preprocessing()
    # 전처리된 데이터프레임 출력
    print(ethereum_price)


  0%|          | 0/100 [00:00<?, ?it/s]

2012-04-06 00:00:00
2020-05-12 09:01:00


  2%|▏         | 2/100 [00:00<00:13,  7.09it/s]

2023-01-04 09:01:00
            eth_price   Symbol
eth_date                      
2017-08-17     302.00  ETHUSDT
2017-08-18     293.96  ETHUSDT
2017-08-19     290.91  ETHUSDT
2017-08-20     299.10  ETHUSDT
2017-08-21     323.29  ETHUSDT
...               ...      ...
2022-12-31    1196.13  ETHUSDT
2023-01-01    1200.34  ETHUSDT
2023-01-02    1214.00  ETHUSDT
2023-01-03    1214.55  ETHUSDT
2023-01-04    1250.08  ETHUSDT

[1967 rows x 2 columns]
